In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password

In [2]:
user

'postgres'

### Extract CSVs into DataFrames

In [3]:
recipe_file = "Resources/RAW_recipes.csv"
df_recipe = pd.read_csv(recipe_file)
df_recipe.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
interactions_file = "Resources/RAW_interactions.csv"
df_interactions = pd.read_csv(interactions_file)
df_interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [5]:
df_recipe = df_recipe.drop(columns = ['contributor_id', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients', 'steps', 'nutrition'])

In [14]:
df_recipe.head()

,name,id,minutes,submitted,nutrition
0,arriba baked winter squash mexican style,137739,55,2005-09-16,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]"
1,a bit different breakfast pizza,31490,30,2002-06-17,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]"
2,all in the kitchen chili,112140,130,2005-02-25,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]"
3,alouette potatoes,59389,45,2003-04-14,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]"
4,amish tomato ketchup for canning,44061,190,2002-10-25,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]"


### Transform premise DataFrame

In [5]:
premise_df['ID'] = premise_df.index
premise_df.head()

,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code,ID
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0,1
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1,2
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1,3
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2,4


In [6]:
new_premise_df = premise_df[['License Serial Number', 'Premises Name', 'County ID Code']].copy()
new_premise_df = new_premise_df.rename(columns={'License Serial Number':'id', 'Premises Name':'premises_name', 'County ID Code':'county_id_code'})
new_premise_df.head()

,id,premises_name,county_id_code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2


In [7]:

new_premise_df.drop_duplicates('id', inplace=True)
new_premise_df.set_index('id')

,premises_name,county_id_code
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1
2213240,COOPERSTOWN BERT CORP,3
2213242,DALE B HOLDERMAN,4
1300091,TWO THOUSAND FIFTEEN ARTISANAL LLC,2
1310030,NEW FUZHOU STYLE INC,1


In [8]:
new_premise_df.head()

,id,premises_name,county_id_code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2
5,1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [9]:
new_county_df = county_df[['ID', 'County Name (Licensee)', 'License Count', 'County ID Code']].copy()
new_county_df = new_county_df.rename(columns={'ID':'id', 'County Name (Licensee)':'county_name', 'License Count':'license_count', 'County ID Code':'county_id_code'})
new_county_df.head()

,id,county_name,license_count,county_id_code
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41


In [10]:
new_county_df.set_index('id', inplace=True)
new_county_df.head()

,county_name,license_count,county_id_code
id,,,
0,ALBANY,77,5
1,ALLEGANY,4,59
2,BRONX,104,0
3,BROOME,14,35
4,CATTARAUGUS,9,41


### Create database connection

In [11]:
rds_connection_string = user + ':' + password + '@localhost:5432/License_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
# Confirm tables
engine.table_names()

['premise_data', 'county_data']

### Load DataFrames into database

In [13]:
new_premise_df.to_sql('premise_data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "premise_data_pkey"
DETAIL:  Key (id)=(1311660) already exists.

[SQL: INSERT INTO premise_data (id, premises_name, county_id_code) VALUES (%(id)s, %(premises_name)s, %(county_id_code)s)]
[parameters: ({'id': 1311660, 'premises_name': 'CANA ARRIBA GROCERY NO 2 INC', 'county_id_code': 0}, {'id': 1310023, 'premises_name': 'JOHANA GROCERY & DELI CORP', 'county_id_code': 0}, {'id': 1310024, 'premises_name': 'TKO BEVERAGES LLC', 'county_id_code': 1}, {'id': 1311663, 'premises_name': '181 LEXINGTON AVENUE BBQ LLC', 'county_id_code': 2}, {'id': 1310029, 'premises_name': 'AZIZ DELI & GRILL CORP', 'county_id_code': 1}, {'id': 2213240, 'premises_name': 'COOPERSTOWN BERT CORP', 'county_id_code': 3}, {'id': 2213242, 'premises_name': 'DALE B HOLDERMAN', 'county_id_code': 4}, {'id': 1300091, 'premises_name': 'TWO THOUSAND FIFTEEN ARTISANAL LLC', 'county_id_code': 2}  ... displaying 10 of 1839 total bound

In [14]:
new_county_df.to_sql('county_data', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "id" violates not-null constraint
DETAIL:  Failing row contains (null, ALBANY, 77, 5).

[SQL: INSERT INTO county_data (county_name, license_count, county_id_code) VALUES (%(county_name)s, %(license_count)s, %(county_id_code)s)]
[parameters: ({'county_name': 'ALBANY', 'license_count': 77, 'county_id_code': 5}, {'county_name': 'ALLEGANY', 'license_count': 4, 'county_id_code': 59}, {'county_name': 'BRONX', 'license_count': 104, 'county_id_code': 0}, {'county_name': 'BROOME', 'license_count': 14, 'county_id_code': 35}, {'county_name': 'CATTARAUGUS', 'license_count': 9, 'county_id_code': 41}, {'county_name': 'CAYUGA', 'license_count': 10, 'county_id_code': 44}, {'county_name': 'CHAUTAUQUA', 'license_count': 10, 'county_id_code': 45}, {'county_name': 'CHEMUNG', 'license_count': 7, 'county_id_code': 37}  ... displaying 10 of 62 total bound parameter sets ...  {'county_name': 'WYOMING', 'license_count': 2, 'county_id_code'